# Import Required Libraries
Import necessary libraries for UI creation (tkinter), image processing (PIL, OpenCV), and deep learning (PyTorch).

In [1]:
# Import necessary libraries for UI creation, image processing, and deep learning

import tkinter as tk  # For creating the UI
from tkinter import filedialog, messagebox  # For file selection and alerts
from PIL import Image, ImageTk  # For image processing and displaying in the UI
import cv2  # For image manipulation and OpenCV operations
import torch  # For deep learning model handling
import torchvision.transforms as transforms  # For preprocessing images for the model
import numpy as np  # For numerical operations

# Define UI Components
Create the UI components including image upload area, detection controls, and result display using tkinter or ipywidgets.

In [2]:
# Define the UI components for testing the object detection model

class ObjectDetectionUI:
    def __init__(self, root, model, class_names):
        self.root = root
        self.model = model
        self.class_names = class_names
        self.image = None
        self.processed_image = None

        # Configure the main window
        self.root.title("Object Detection Model Tester")
        self.root.geometry("800x600")
        self.root.configure(bg="#f0f0f0")

        # Create UI components
        self.create_image_upload_area()
        self.create_detection_controls()
        self.create_result_display()

    def create_image_upload_area(self):
        """Create the image upload area."""
        frame = tk.Frame(self.root, bg="#f0f0f0", pady=10)
        frame.pack(fill=tk.X)

        upload_button = tk.Button(
            frame, text="Upload Image", command=self.upload_image, bg="#4CAF50", fg="white", padx=10, pady=5
        )
        upload_button.pack(side=tk.LEFT, padx=10)

        self.image_label = tk.Label(frame, text="No image uploaded", bg="#f0f0f0", fg="#555")
        self.image_label.pack(side=tk.LEFT, padx=10)

    def create_detection_controls(self):
        """Create the detection controls."""
        frame = tk.Frame(self.root, bg="#f0f0f0", pady=10)
        frame.pack(fill=tk.X)

        detect_button = tk.Button(
            frame, text="Run Detection", command=self.run_detection, bg="#2196F3", fg="white", padx=10, pady=5
        )
        detect_button.pack(side=tk.LEFT, padx=10)

        self.confidence_label = tk.Label(frame, text="Confidence Threshold:", bg="#f0f0f0", fg="#555")
        self.confidence_label.pack(side=tk.LEFT, padx=10)

        self.confidence_slider = tk.Scale(
            frame, from_=0.1, to=1.0, resolution=0.1, orient=tk.HORIZONTAL, bg="#f0f0f0", fg="#555"
        )
        self.confidence_slider.set(0.5)
        self.confidence_slider.pack(side=tk.LEFT, padx=10)

    def create_result_display(self):
        """Create the result display area."""
        frame = tk.Frame(self.root, bg="#f0f0f0", pady=10)
        frame.pack(fill=tk.BOTH, expand=True)

        self.canvas = tk.Canvas(frame, bg="#ffffff", width=600, height=400)
        self.canvas.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def upload_image(self):
        """Handle image upload."""
        file_path = filedialog.askopenfilename(
            title="Select an Image", filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_path:
            self.image = Image.open(file_path)
            self.image_label.config(text=f"Uploaded: {file_path.split('/')[-1]}")
            self.display_image(self.image)

    def display_image(self, image):
        """Display the uploaded image on the canvas."""
        self.processed_image = image.resize((600, 400), Image.ANTIALIAS)
        self.tk_image = ImageTk.PhotoImage(self.processed_image)
        self.canvas.create_image(300, 200, image=self.tk_image, anchor=tk.CENTER)

    def run_detection(self):
        """Run object detection on the uploaded image."""
        if self.image is None:
            messagebox.showerror("Error", "Please upload an image first.")
            return

        # Preprocess the image
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_image = transform(self.image).unsqueeze(0)

        # Run the model
        with torch.no_grad():
            outputs = self.model(input_image)

        # Process the outputs
        self.display_results(outputs)

    def display_results(self, outputs):
        """Display the detection results on the canvas."""
        self.canvas.delete("all")
        self.canvas.create_image(300, 200, image=self.tk_image, anchor=tk.CENTER)

        for output in outputs[0]:
            x1, y1, x2, y2, conf, class_id = output
            if conf >= self.confidence_slider.get():
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                self.canvas.create_rectangle(x1, y1, x2, y2, outline="red", width=2)
                self.canvas.create_text(
                    x1, y1 - 10, text=f"{self.class_names[int(class_id)]} ({conf:.2f})", fill="red", anchor=tk.W
                )


# Example usage:
# root = tk.Tk()
# model = ...  # Load your PyTorch model here
# class_names = ["class1", "class2", "class3"]  # Replace with your class names
# app = ObjectDetectionUI(root, model, class_names)
# root.mainloop()

# Load Model
Define functions to load and prepare the pre-trained object detection model. Include options to load different models (custom model and YOLO).

In [3]:
def load_model(model_type="custom", model_path=None):
    """
    Load and prepare the object detection model.

    Parameters:
        model_type (str): Type of model to load ("custom" or "yolo").
        model_path (str): Path to the custom model file (if applicable).

    Returns:
        model: Loaded PyTorch model.
        class_names (list): List of class names for the model.
    """
    if model_type == "custom":
        if model_path is None:
            raise ValueError("Please provide the path to the custom model.")
        try:
            # Load the custom model
            model = torch.load(model_path, map_location=torch.device("cpu"))
            model.eval()
            print(f"✅ Custom model loaded from {model_path}")
            # Define class names (update based on your dataset)
            class_names = ["class1", "class2", "class3"]
        except Exception as e:
            raise RuntimeError(f"Failed to load custom model: {e}")
    elif model_type == "yolo":
        try:
            # Load a pre-trained YOLO model from torchvision
            model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)
            model.eval()
            print("✅ YOLO model loaded successfully")
            # Define class names for YOLO
            class_names = model.names
        except Exception as e:
            raise RuntimeError(f"Failed to load YOLO model: {e}")
    else:
        raise ValueError("Invalid model type. Choose 'custom' or 'yolo'.")

    return model, class_names

# Create Detection Logic
Implement the core detection functionality that processes images, runs inference with the model, and extracts bounding boxes with confidence scores.

In [4]:
def detect_objects(image, model, confidence_threshold=0.5):
    """
    Detect objects in an image using the provided model.

    Parameters:
        image (PIL.Image): The input image.
        model (torch.nn.Module): The object detection model.
        confidence_threshold (float): Minimum confidence score to consider a detection.

    Returns:
        list: A list of detections, where each detection is a tuple (x1, y1, x2, y2, confidence, class_id).
    """
    # Preprocess the image
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    input_image = transform(image).unsqueeze(0)

    # Run the model
    with torch.no_grad():
        outputs = model(input_image)

    # Process the outputs
    detections = []
    for output in outputs[0]:  # Assuming the model returns a list of detections
        x1, y1, x2, y2, conf, class_id = output
        if conf >= confidence_threshold:
            detections.append((x1, y1, x2, y2, conf, class_id))

    return detections

# Build and Launch the UI
Assemble all UI components and connect them to the detection logic. Set up event handlers for user interactions.

In [ ]:
# Build and Launch the UI

if __name__ == "__main__":
    # Initialize the main Tkinter window
    root = tk.Tk()

    # Load the model and class names
    try:
        # Replace 'custom_model.pth' with the path to your model file
        model, class_names = load_model(model_type="custom", model_path="custom_model.pth")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load model: {e}")
        root.destroy()
        exit()

    # Create the Object Detection UI
    app = ObjectDetectionUI(root, model, class_names)

    # Start the Tkinter event loop
    root.mainloop()

NameError: name 'model' is not defined

: 

# Add Visualization Functions
Create functions to visualize detection results by drawing bounding boxes and labels on images with appropriate color coding based on confidence.

In [ ]:
def visualize_detections(image, detections, class_names, confidence_threshold=0.5):
    """
    Visualize detection results by drawing bounding boxes and labels on the image.

    Parameters:
        image (PIL.Image): The input image.
        detections (list): List of detections, where each detection is a tuple (x1, y1, x2, y2, confidence, class_id).
        class_names (list): List of class names corresponding to class IDs.
        confidence_threshold (float): Minimum confidence score to display a detection.

    Returns:
        PIL.Image: The image with bounding boxes and labels drawn.
    """
    # Convert the image to a NumPy array for OpenCV processing
    image_np = np.array(image)

    # Iterate through detections and draw bounding boxes and labels
    for x1, y1, x2, y2, conf, class_id in detections:
        if conf >= confidence_threshold:
            # Convert coordinates to integers
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Define the color for the bounding box (based on class ID)
            color = (0, 255, 0)  # Green for high confidence
            if conf < 0.75:
                color = (0, 255, 255)  # Yellow for medium confidence
            if conf < 0.5:
                color = (0, 0, 255)  # Red for low confidence

            # Draw the bounding box
            cv2.rectangle(image_np, (x1, y1), (x2, y2), color, 2)

            # Prepare the label text
            label = f"{class_names[int(class_id)]}: {conf:.2f}"

            # Calculate text size and position
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            label_y = max(y1 - text_height - baseline, 0)

            # Draw a filled rectangle for the label background
            cv2.rectangle(image_np, (x1, label_y), (x1 + text_width, label_y + text_height + baseline), color, -1)

            # Put the label text
            cv2.putText(image_np, label, (x1, label_y + text_height), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Convert the image back to PIL format
    return Image.fromarray(image_np)